## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os 
import time


pd.set_option('display.max_columns', None)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Adjust the longitude from 0 to 360 to -180 to 180
def adjust_longitude(lon):
    lon_adjusted = ((lon + 180) % 360) - 180
    lon_adjusted[lon_adjusted == -180] = 180
    return lon_adjusted

## ERA5-Land 2m Temperature Dataset

In [ ]:
# Open/load dataset
path_era5 = '/Users/alexaterrazas/Desktop/Lake Transfer Function Manuscript/ERA5 data/'
ds_tas = xr.open_dataset(path_era5 + 'tas_data.nc')

# Subset the data for the time range from 2013 to 2021
tas = ds_tas['t2m'].sel(time=slice('2013-01-01', '2021-12-01'))

# Calculate the monthly mean
tas_monthly_mean = tas.groupby('time.month').mean(dim='time')

# Convert from Kelvin to Celsius
tas_monthly_mean = tas_monthly_mean - 273.15

# Apply the adjust_longitude function
tas_monthly_mean.coords['longitude'] = adjust_longitude(tas_monthly_mean.coords['longitude'])

# Sort by longitude to ensure the dataset is correctly ordered
tas_monthly_mean = tas_monthly_mean.sortby('longitude')

tas_monthly_mean


## ERA5 Total Cloud Cover Dataset

In [ ]:
# Open/load dataset
path_era5 = '/Users/alexaterrazas/Desktop/Lake Transfer Function Manuscript/ERA5 data/'
ds_tcc = xr.open_dataset(path_era5 + 'tcc_data.nc')

# Subset the data for the time range from 2013 to 2021
tcc = ds_tcc['tcc'].sel(time=slice('2013-01-01', '2021-12-01'))

# Convert to percentage
tcc_percentage = tcc * 100

# Calculate the monthly mean
tcc_monthly_mean = tcc_percentage.groupby('time.month').mean(dim='time')

# Apply the adjust_longitude function
tcc_monthly_mean.coords['longitude'] = adjust_longitude(tcc_monthly_mean.coords['longitude'])

# Sort by longitude to ensure the dataset is correctly ordered
tcc_monthly_mean = tcc_monthly_mean.sortby('longitude')


tcc_monthly_mean

## ERA5-Land 2m Dewpoint Temperature

### Dew Point Temperature

In [ ]:
# Open/load dataset
path_era5 = '/Users/alexaterrazas/Desktop/Lake Transfer Function Manuscript/ERA5 data/'
ds_d2m_sp = xr.open_dataset(path_era5 + 'd2m_sp_data.nc')

# Subset the data for the time range from 2013 to 2021
d2m = ds_d2m_sp['d2m'].sel(time=slice('2013-01-01', '2021-12-01'))

# Calculate the monthly mean
d2m_monthly_mean = d2m.groupby('time.month').mean(dim='time')

# Convert from Kelvin to Celsius
d2m_monthly_mean = d2m_monthly_mean - 273.15

# Convert from Kelvin to Celsius
d2m_monthly_mean = d2m_monthly_mean

# Apply the adjust_longitude function
d2m_monthly_mean.coords['longitude'] = adjust_longitude(d2m_monthly_mean.coords['longitude'])

# Sort by longitude to ensure the dataset is correctly ordered
d2m_monthly_mean = d2m_monthly_mean.sortby('longitude')
d2m_monthly_mean

## Calculate Relative Humidity

In [ ]:
def saturation_vapor_pressure(T):
    """
    Calculate the saturation vapor pressure using the Clausius-Clapeyron equation.
    
    Parameters:
    T (float): Temperature in Kelvin.
    
    Returns:
    float: Saturation vapor pressure in hPa.
    """
    e0 = 6.112  # hPa
    Lv = 2.5e6  # J/kg
    Rv = 461.5  # J/(kg·K)
    T0 = 273.15  # K
    
    es = e0 * np.exp((Lv / Rv) * (1 / T0 - 1 / T))
    
    return es

def calculate_relative_humidity(T, Td):
    """
    Calculate the relative humidity given temperature (T) and dewpoint temperature (Td).
    
    Parameters:
    T (float): Temperature in degrees Celsius.
    Td (float): Dewpoint temperature in degrees Celsius.
    
    Returns:
    float: Relative humidity as a percentage.
    """
    # Convert temperatures from Celsius to Kelvin
    T_kelvin = T + 273.15
    Td_kelvin = Td + 273.15
    
    # Calculate saturation vapor pressures
    es_T = saturation_vapor_pressure(T_kelvin)
    es_Td = saturation_vapor_pressure(Td_kelvin)
    
    # Calculate relative humidity
    RH = 100 * (es_Td / es_T)
    
    return RH

# Use calculate_relative_humidity function
T = tas_monthly_mean    # Temperature in degrees Celsius
Td = d2m_monthly_mean   # Dewpoint temperature in degrees Celsius

# Use calculate_relative_humidity function
RH_monthly_mean = calculate_relative_humidity(T, Td)
RH_monthly_mean

## ERA5-Land Surface net solar radiation

In [ ]:
# Open/load dataset
path_era5 = '/Users/alexaterrazas/Desktop/Lake Transfer Function Manuscript/ERA5 data/'
ds_ssr = xr.open_dataset(path_era5 + 'ssr_data.nc')

# Subset the data for the time range from 2013 to 2021
ssr = ds_ssr['ssr'].sel(time=slice('2013-01-01', '2021-12-01'))

# Calculate the monthly mean
ssr_monthly_mean = ssr.groupby('time.month').mean(dim='time')


# Apply the adjust_longitude function
ssr_monthly_mean.coords['longitude'] = adjust_longitude(ssr_monthly_mean.coords['longitude'])

# Sort by longitude to ensure the dataset is correctly ordered
ssr_monthly_mean = ssr_monthly_mean.sortby('longitude')

ssr_monthly_mean


## ERA5-Land 10m horizontal wind component

In [ ]:
# Open/load dataset
path_era5 = '/Users/alexaterrazas/Desktop/Lake Transfer Function Manuscript/ERA5 data/'
ds_u10 = xr.open_dataset(path_era5 + 'u10_data.nc')

# Subset the data for the time range from 2013 to 2021
u10 = ds_u10['u10'].sel(time=slice('2013-01-01', '2021-12-01'))

# Calculate the monthly mean
u10_monthly_mean = u10.groupby('time.month').mean(dim='time')

# Apply the adjust_longitude function
u10_monthly_mean.coords['longitude'] = adjust_longitude(u10_monthly_mean.coords['longitude'])

# Sort by longitude to ensure the dataset is correctly ordered
u10_monthly_mean = u10_monthly_mean.sortby('longitude')

u10_monthly_mean


## LakeTemp Dataset

In [ ]:
#open dataset
path_laketemp = '/Users/alexaterrazas/Desktop/Lake Transfer Function Manuscript/LakeTEMP_v1/'
lake_data_agg = pd.read_csv(path_laketemp + 'LakeTEMP_aggregated_v1.csv')
lake_data_agg

#remove reservoirs
    #1 = lake
    #2 = reservoir
    #3 = controlled lake (natural lake with regulation structure)
mask_nonlake = lake_data_agg['Lake_type'].isin([1])
lake_data = lake_data_agg[mask_nonlake]
print(len(lake_data_agg), len(lake_data))

#remove uncertain or unreliable data
    # relaiable and acceptable >= 46 observations
mask_obs = lake_data['n_obs'] >= 46
lake_data = lake_data[mask_obs]
print(len(lake_data))


#assess intermittency and remove lakes that are permanently dry
    #3 = 100 % ‘land’ + ‘unknown’ observations (Permanently dry lake with seasonal snow or ice cover)
    #4 = 100 % ‘land’ observations             (Permanently dry lake; or wetland with dense vegetation)
mask_percentland = lake_data['intermittency'].isin([0,1,2])
lake_data = lake_data[mask_percentland]

#lake surface area can't be smaller than the resolution of the dataset
lake_data = lake_data[lake_data['Lake_area'] > 81]
lake_data

## Caculate seasonal lake water temperatures

In [ ]:
# Function to calculate seasonal average based on hemisphere
def calculate_seasonal_avg(df, nh_months, sh_months):
    seasonal_avg = np.where(df['center_lat'] >= 0, df[nh_months].mean(axis=1), df[sh_months].mean(axis=1))
    return seasonal_avg

In [ ]:
# Calculate the mean lake surface water temperature for all 12 months
lake_data['lswt_ann_avg'] = lake_data[['Tmonth_mean_1', 'Tmonth_mean_2', 'Tmonth_mean_3', 'Tmonth_mean_4', 
                             'Tmonth_mean_5', 'Tmonth_mean_6', 'Tmonth_mean_7', 'Tmonth_mean_8', 
                             'Tmonth_mean_9', 'Tmonth_mean_10', 'Tmonth_mean_11', 'Tmonth_mean_12']].mean(axis=1)


# Calculate the seasonal average for NH April-October (A-O) and SH October-April (A-O)
lake_data['lswt_ao_avg'] = calculate_seasonal_avg(lake_data, 
                                                   ['Tmonth_mean_4', 'Tmonth_mean_5', 'Tmonth_mean_6', 'Tmonth_mean_7', 'Tmonth_mean_8', 'Tmonth_mean_9', 'Tmonth_mean_10'],
                                                   ['Tmonth_mean_10', 'Tmonth_mean_11', 'Tmonth_mean_12', 'Tmonth_mean_1', 'Tmonth_mean_2', 'Tmonth_mean_3', 'Tmonth_mean_4'])

# Calculate the seasonal average for NH April-May-June (AMJ) and SH October-November-December (OND)
lake_data['lswt_amj_avg'] = calculate_seasonal_avg(lake_data, 
                                         ['Tmonth_mean_4', 'Tmonth_mean_5', 'Tmonth_mean_6'],
                                         ['Tmonth_mean_10', 'Tmonth_mean_11', 'Tmonth_mean_12'])

# Calculate the seasonal average for NH June-July-August (JJA) and SH December-January-February (DJF)
lake_data['lswt_jja_avg'] = calculate_seasonal_avg(lake_data, 
                                         ['Tmonth_mean_6', 'Tmonth_mean_7', 'Tmonth_mean_8'],
                                         ['Tmonth_mean_12', 'Tmonth_mean_1', 'Tmonth_mean_2'])

# Find the warmest month for each row and create a new column 'warmest_avg'
lake_data['lswt_warmest_avg'] = lake_data[['Tmonth_mean_1', 'Tmonth_mean_2', 'Tmonth_mean_3', 'Tmonth_mean_4', 
                            'Tmonth_mean_5', 'Tmonth_mean_6', 'Tmonth_mean_7', 'Tmonth_mean_8', 
                            'Tmonth_mean_9', 'Tmonth_mean_10', 'Tmonth_mean_11', 'Tmonth_mean_12']].max(axis=1)

#convert lake_data to Celcius
lake_data['lswt_ann_avg']  = (lake_data['lswt_ann_avg']/100)
lake_data['lswt_ao_avg']  = (lake_data['lswt_ao_avg']/100)
lake_data['lswt_amj_avg']  = (lake_data['lswt_amj_avg']/100)
lake_data['lswt_jja_avg']  = (lake_data['lswt_jja_avg']/100)
lake_data['lswt_warmest_avg']  = (lake_data['lswt_warmest_avg']/100)


# Print the first few rows to verify the new columns
lake_data[['center_long', 'center_lat', 'lswt_ann_avg', 'lswt_ao_avg', 'lswt_amj_avg', 'lswt_jja_avg', 'lswt_warmest_avg']].head()


In [ ]:
# Make a new column 'abs_lat' with the absolute values of 'center_lat'
lake_data['abs_lat'] = abs(lake_data['center_lat'])

In [ ]:
# Make a new column for seasonal LSWT^2
lake_data['sq_lswt_ann'] = lake_data['lswt_ann_avg']**2
lake_data['sq_lswt_ao'] = lake_data['lswt_ao_avg']**2
lake_data['sq_lswt_amj'] = lake_data['lswt_amj_avg']**2
lake_data['sq_lswt_jja'] = lake_data['lswt_jja_avg']**2
lake_data['sq_lswt_warmest'] = lake_data['lswt_warmest_avg']**2

# Transform lake area and depth
lake_data['log_lake_area'] = np.log(lake_data['Lake_area'])
lake_data['log_depth'] = np.log(lake_data['Depth_avg'])

# Covert elevation from meters to km
lake_data['elevation_km'] = lake_data['Elevation']/1000


lake_data

## Find nearest values at the LakeTemp lake coordinates for variables: tas, tcc, rh, and u10 and caculate seasonal and mean annual averages

In [ ]:
# # Verify the coordinate range
# print(f"Lake data longitude range: {lake_data['center_long'].min()} to {lake_data['center_long'].max()}")
# print(f"Lake data latitude range: {lake_data['center_lat'].min()} to {lake_data['center_lat'].max()}")

# print(f"ERA5 data longitude range: {tas_monthly_mean.coords['longitude'].min().values} to {tas_monthly_mean.coords['longitude'].max().values}")
# print(f"ERA5 data latitude range: {tas_monthly_mean.coords['latitude'].min().values} to {tas_monthly_mean.coords['latitude'].max().values}")

# Load the datasets (assuming they are already loaded as tas_monthly_mean, tcc_monthly_mean, RH_monthly_mean, u10_monthly_mean)
tas_monthly_mean.load()
tcc_monthly_mean.load()
RH_monthly_mean.load()
ssr_monthly_mean.load()
u10_monthly_mean.load()

# Prepare the coordinates from the LakeTemp data DataFrame
coordinates = lake_data[['center_long', 'center_lat']].values

# Initialize dictionaries to store the nearest values for each month for all datasets
nearest_values = {var: {month: [] for month in range(1, 13)} for var in ['temp', 'tcc', 'rh', 'ssr', 'u10']}

print("Initialized nearest values dictionaries")

# Convert the coordinates to an xarray DataArray for vectorized operations
coords_da = xr.DataArray(coordinates, dims=['points', 'coord'], coords={'coord': ['longitude', 'latitude']})

# Loop through each month and find the nearest values for all datasets at once
for month in range(1, 13):
    nearest_values['temp'][month] = tas_monthly_mean.sel(month=month).sel(longitude=coords_da.sel(coord='longitude'), 
                                                                          latitude=coords_da.sel(coord='latitude'), 
                                                                          method='nearest').values
    nearest_values['tcc'][month] = tcc_monthly_mean.sel(month=month).sel(longitude=coords_da.sel(coord='longitude'), 
                                                                         latitude=coords_da.sel(coord='latitude'), 
                                                                         method='nearest').values
    nearest_values['rh'][month] = RH_monthly_mean.sel(month=month).sel(longitude=coords_da.sel(coord='longitude'), 
                                                                       latitude=coords_da.sel(coord='latitude'), 
                                                                       method='nearest').values
    nearest_values['ssr'][month] = ssr_monthly_mean.sel(month=month).sel(longitude=coords_da.sel(coord='longitude'), 
                                                                         latitude=coords_da.sel(coord='latitude'), 
                                                                         method='nearest').values
    nearest_values['u10'][month] = u10_monthly_mean.sel(month=month).sel(longitude=coords_da.sel(coord='longitude'), 
                                                                         latitude=coords_da.sel(coord='latitude'), 
                                                                         method='nearest').values
    print(f"Month {month}: Processed nearest values for all datasets")

# Add the lists of nearest values as columns to the DataFrame
for var in ['temp', 'tcc', 'rh', 'ssr', 'u10']:
    for month in range(1, 13):
        lake_data[f'nearest_{var}_{month}'] = nearest_values[var][month]

# Drop NaN values in each of the nearest variables
for month in range(1, 13):
    lake_data = lake_data.dropna(subset=[f'nearest_temp_{month}', f'nearest_tcc_{month}', f'nearest_rh_{month}', f'nearest_ssr_{month}', f'nearest_u10_{month}'])


# Calculate the mean temperature for all 12 months
lake_data['tas_ann_avg'] = lake_data[[f'nearest_temp_{month}' for month in range(1, 13)]].mean(axis=1)

# Calculate the seasonal average for NH April-October (A-O) and SH October-April (A-O)
lake_data['tas_ao_avg'] = calculate_seasonal_avg(lake_data, 
                                                 [f'nearest_temp_{month}' for month in [4, 5, 6, 7, 8, 9, 10]],
                                                 [f'nearest_temp_{month}' for month in [10, 11, 12, 1, 2, 3, 4]])

# Calculate the seasonal average for NH April-May-June (AMJ) and SH October-November-December (OND)
lake_data['tas_amj_avg'] = calculate_seasonal_avg(lake_data, 
                                                  [f'nearest_temp_{month}' for month in [4, 5, 6]],
                                                  [f'nearest_temp_{month}' for month in [10, 11, 12]])

# Calculate the seasonal average for NH June-July-August (JJA) and SH December-January-February (DJF)
lake_data['tas_jja_avg'] = calculate_seasonal_avg(lake_data, 
                                                  [f'nearest_temp_{month}' for month in [6, 7, 8]],
                                                  [f'nearest_temp_{month}' for month in [12, 1, 2]])

# Find the warmest month for each row and create a new column 'tas_warmest_month'
lake_data['tas_warmest_month'] = lake_data[[f'nearest_temp_{month}' for month in range(1, 13)]].max(axis=1)

# Calculate the mean values for TCC, RH, and U10 for all 12 months
for var in ['tcc', 'rh', 'ssr', 'u10']:
    lake_data[f'{var}_ann_avg'] = lake_data[[f'nearest_{var}_{month}' for month in range(1, 13)]].mean(axis=1)

# Print the first few rows to verify the new columns
lake_data[['center_long', 'center_lat', 
                 'tas_ann_avg', 'tas_ao_avg', 'tas_amj_avg', 'tas_jja_avg', 'tas_warmest_month',
                 'tcc_ann_avg', 'rh_ann_avg', 'ssr_ann_avg', 'u10_ann_avg']].head()

## Save lake_data df as a csv

In [ ]:
lake_data.to_csv('ERA5_LakeTemp.csv', index=False)